In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [5]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [6]:
type(train_dataset)

torchvision.datasets.mnist.MNIST

In [16]:
max(train_dataset.targets).item()

9

In [17]:
min(train_dataset.targets).item()

0

In [8]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [33]:
dik.values()

dict_values(['five', 'three', 'two', 'four', 'six', 'seven', 'eight', 'nine'])

In [46]:
dik={1: 'zero', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'}
for i in range(len(dik)):
    print(dik.get(i))

None
one
two
three
four
five
six
seven
eight


In [80]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes,arch):
        super().__init__()
        layerss=[]
        sizes=[]
        for l, v in arch:
            layerss.append(l)
            sizes.append(v)
        
        self.layers=nn.ModuleList()
        prev_size= sizes[0]
        
        #Input layer
        self.layers.append(nn.Linear(input_size,prev_size))
        #Hidden layers
        for i in range(1,len(arch)):
            # print(i)
            size=sizes[i]
            layer=layerss[i]
            if layer == 'linear':
                self.layers.append(nn.Linear(prev_size,size))
            elif layer == 'relu':
                self.layers.append(nn.ReLU())
            elif layer =='sigmoid':
                self.layers.append(nn.Sigmoid())
            elif layer == 'batchnorm1d':
                self.layers.append(nn.BatchNorm1d(prev_size))
            elif layer == 'dropout':
                self.layers.append(nn.Dropout())
            elif layer == 'flatten':
                self.layers.append(nn.Flatten())
            if size!=None:
                prev_size=size
        #Output layer
        self.layers.append(nn.Linear(hidden_size,num_classes))
        self.softmax=nn.Softmax(dim=1)
        
    def forward(self, x):
        for layer in self.layers:
            out = layer(x)
            out = self.softmax(out)
        return out

In [81]:
layers_list = [
    ('linear', 128),
    ('relu', None),
    ('linear', 64),
    ('relu', None),
    ('linear', 32),
    ('relu', None),
    ('linear', 16),
    ('relu', None),
    ('linear', 8),
    ('relu', None),
    ('linear', 4),
    ('relu', None),
    ('linear', 2),
    ('relu', None),
    ('linear', 1),
    ('relu', None)
]

In [82]:
model = NN(784, 128, 10,arch=layers_list).to(device)
model

NN(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ReLU()
    (8): Linear(in_features=16, out_features=8, bias=True)
    (9): ReLU()
    (10): Linear(in_features=8, out_features=4, bias=True)
    (11): ReLU()
    (12): Linear(in_features=4, out_features=2, bias=True)
    (13): ReLU()
    (14): Linear(in_features=2, out_features=1, bias=True)
    (15): ReLU()
    (16): Linear(in_features=128, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)